## Предсказание свободной энергии связывания

В этой практике вы реализуете собственную графовую архитектуру для предсказания свободной энергии связывания двух белков, которая будет более точно учитывать их геометрию, но сохранит инвариантность относительно движений в пространстве.

На практике сделали всю подготовительную работу для проведения экспериментов, а также обнаружили, что в случае простой графовой модели лучший результат дал граф, построенный на атомной структуре интерфейса, но лишённый внутримолекулярных связей, т.е. рёбер, соединяющих атомы одной и той же молекулы.

Однако, наша модель была крайне простой, и в своих экспериментах вы можете обнаружить, что другой представление входных данных в сочетании с более сложной архитектурой сработает ещё лучше. В качестве бонусного задания вы сможете провести любые эксперименты с архитектурой и способом представления данных.

#### Подготовка данных (с практики по GNN)

In [1]:
import json
from pathlib import Path
from typing import Any, Protocol, Type

import torch
import torch.nn.functional as F
from torch import Tensor, nn
from torch.optim import Adam
from torch.utils.data import Dataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import radius_graph
from torch_geometric.nn.conv import (
    GATConv,
    GatedGraphConv,
    GCNConv,
    GraphConv,
    MessagePassing,
)
from torch_geometric.nn.pool import global_add_pool, global_max_pool, global_mean_pool
from torch_geometric.utils import to_undirected

In [2]:
# fmt: off
ATOM_NAMES = [
    "C", "CA", "CB", "CD", "CD1", "CD2", "CE", "CE1", "CE2", "CE3", "CG", "CG1", "CG2", "CH2", "CZ", "CZ2", "CZ3",
    "H", "H2", "H3", "HA", "HA2", "HA3", "HB", "HB1", "HB2", "HB3", "HD1", "HD11", "HD12", "HD13",
    "HD2", "HD21", "HD22", "HD23", "HD3", "HE", "HE1", "HE2", "HE21", "HE22", "HE3",
    "HG", "HG1", "HG11", "HG12", "HG13", "HG2", "HG21", "HG22", "HG23", "HG3", "HH", "HH11", "HH12",
    "HH2", "HH21", "HH22", "HZ", "HZ1", "HZ2", "HZ3",
    "N", "ND1", "ND2", "NE", "NE1", "NE2", "NH1", "NH2", "NZ",
    "O", "OD1", "OD2", "OE1", "OE2", "OG", "OG1", "OH", "OXT", "SD", "SG",
]
# fmt: on
ATOMS_INDICES = {x: i for i, x in enumerate(ATOM_NAMES)}
RESIDUES = "ACDEFGHIKLMNPQRSTVWY"
RESIDUE_INDICES = {c: i for i, c in enumerate(RESIDUES)}
ATOM_COLORS = {
    "C": "gray",
    "N": "blue",
    "O": "red",
    "H": "black",
    "S": "yellow",
}

In [3]:
class AtomicInterfaceGraph(Protocol):
    atoms: Tensor  # (N): идентификаторы типов атомов
    residues: Tensor  # (N): идентификаторы аминокислот
    is_receptor: Tensor  # (N): 1 для атомов рецептора, 0 для атомов лиганда
    coordinates: Tensor  # (N x 3): координаты атомов
    edge_index: Tensor  # (2 x E) список рёбер между атомами
    distances: Tensor  # (E): расстояния между атомами, соединёнными ребром
    affinity: Tensor | None  # (n): свободная энергия связывания
    batch: Tensor | None  # (N): идентификаторы подграфов в батче, [0, n-1]

In [4]:
def create_interface_graph(
    interface_structure: dict[str, Any],
    graph_radius: float = 4.0,
    n_neighbors: int = 10,
) -> AtomicInterfaceGraph:
    # преобразуем названия атомов в индексы
    encoded_atoms = torch.tensor(
        [
            ATOMS_INDICES.get(atom, len(ATOMS_INDICES))
            for atom in interface_structure["atoms"]
        ]
    )
    # то же для аминокислот
    encoded_residues = torch.tensor(
        [
            RESIDUE_INDICES.get(res, len(RESIDUE_INDICES))
            for res in interface_structure["residues"]
        ]
    )

    is_receptor = torch.tensor(interface_structure["is_receptor"])

    # тензор с координатами атомов
    coordinates = torch.tensor(interface_structure["coords"]).float()

    # используем координаты для построения радиус-графа:
    # NB: модели torch geometric обычно интерпретируют рёбра как направленные,
    # так что мы добавляем обратные рёбра с помощью функции `to_undirected`,
    # если хотим работать с неориентированными графами
    edge_index = to_undirected(
        radius_graph(coordinates, r=graph_radius, max_num_neighbors=n_neighbors)
    )
    # посчитаем расстояния
    src, tgt = edge_index
    distances = torch.linalg.norm(coordinates[src] - coordinates[tgt], dim=1)

    return Data(
        atoms=encoded_atoms,
        residues=encoded_residues,
        is_receptor=is_receptor,
        coordinates=coordinates,
        edge_index=edge_index,
        distances=distances,
        num_nodes=len(encoded_atoms),
    )

In [ ]:
from torch_geometric.transforms.remove_isolated_nodes import RemoveIsolatedNodes


class AtomicGraphDataset(Dataset):
    def __init__(
        self,
        data_json: Path,
        graph_radius: float = 4.0,
        n_neighbors: int = 10,
        remove_intermolecular_edges: bool = False,
    ) -> None:
        self.data: list[AtomicInterfaceGraph] = []
        for x in json.loads(data_json.read_text()):
            item = create_interface_graph(
                x["interface_graph"], graph_radius, n_neighbors
            )
            item.affinity = x["affinity"]
            if remove_intermolecular_edges:
                item = self.remove_intermolecular_edges(item)
            self.data.append(item)

    def __getitem__(self, index: int) -> Data:
        return self.data[index]

    def __len__(self) -> int:
        return len(self.data)

    @staticmethod
    def remove_intermolecular_edges(
        interface_graph: AtomicInterfaceGraph,
    ) -> AtomicInterfaceGraph:
        interface_bigraph = interface_graph.clone()
        src, tgt = interface_bigraph.edge_index
        intermolecular_edges = (
            (interface_graph.is_receptor[src] - interface_graph.is_receptor[tgt])
            .abs()
            .bool()
        )
        interface_bigraph.edge_index = interface_bigraph.edge_index[
            :, intermolecular_edges
        ]

        interface_bigraph = RemoveIsolatedNodes().forward(interface_bigraph)
        src, tgt = interface_bigraph.edge_index
        interface_bigraph.distances = torch.linalg.norm(
            interface_bigraph.coordinates[src] - interface_bigraph.coordinates[tgt],
            dim=1,
        )
        return interface_bigraph

Функция для расчёта метрик

In [6]:
from scipy.stats import pearsonr, spearmanr


@torch.no_grad()
def validate(loader: DataLoader, model: nn.Module) -> tuple[list[float], list[float]]:
    model.eval()
    ys = []
    yhats = []
    loss = 0.0
    for batch in loader:
        yhat = model.forward(batch)
        yhats.extend(yhat.flatten().tolist())
        ys.extend(batch.affinity.tolist())
        loss += F.mse_loss(yhat.flatten(), batch.affinity).item()

    print(f"Loss: {loss / len(ys):.4f}, ", end="")
    print(f"Pearson R: {pearsonr(ys, yhats).statistic:.4f}, ", end="")
    print(f"Spearman R: {spearmanr(ys, yhats).statistic:.4f}")
    model.train()
    return yhats, ys

#### Задание 1 (5 баллов). Реализация E(3)-инвариантной графовой сети

В нашей простой модели мы использовали межатомные расстояния, чтобы построить граф, но далее никакую информацию о геометрии интерфейса не использовали.

Тем не менее, точное относительное положение атомов может существенно определять силу и характер физических взаимодействий.

В этом задании вы реализуете архитектуру графовой сети, которая использует межатомные расстояния при создании сообщений, которыми обмениваются вершины графа. Тем самым результат не будет зависеть от положения и ориентации белкового комплекса в пространстве, но будет явным образом зависеть от геометрии атомных контактов.

Благодаря `pytorch-geometric` реализация таких моделей сравнительно простая, но чтобы не возникло впечатления, что фреймворк делает совсем какую-то магию, перед выполнением задания ознакомьтесь с туториалом по реализации message-passing neural networks: https://pytorch-geometric.readthedocs.io/en/2.5.1/tutorial/create_gnn.html

##### Задание 1.1 (2 балла). E(3)-инвариантный слой графовой сети

Наш слой будет обновлять эмбеддинги вершин в соответствии с уравнением

$h_i^{(t+1)} = \sum_{j \in \mathcal{N}(i)} \text{MLP}^{(t)} \left( \text{concat} (h_i^{(t)}, h_j^{(t)}, e_{ij}) \right)$

т.е. сообщение между вершинами $i$ и $j$ будет формироваться перцептроном, который принимает на вход эмбеддинги вершин и эмбеддинг соединяющего их ребра

Всю работу по распространению сообщений сделает метод `propagate`, вам нужно только реализовать метод `message`, который эти сообщения сформирует

In [ ]:
class InvariantLayer(MessagePassing):
    def __init__(
        self, edge_dim: int, node_dim: int, hidden_dim: int, aggr: str = "sum"
    ) -> None:
        super().__init__(aggr)
        self.message_mlp = nn.Sequential(
            nn.Linear(2 * node_dim + edge_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, node_dim),
        )

    def forward(self, h: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:
        return self.propagate(edge_index, h=h, edge_attr=edge_attr)

    # Ваше решение
    # def message(self, ...) -> ...:
    #     ...

Минимальный тест на работоспособность:

In [ ]:
h = torch.randn(4, 8)
edge_index = torch.tensor([
    [0, 0, 1, 1, 2],
    [1, 3, 2, 3, 3],
])
edge_attr = torch.randn(5, 6)

assert InvariantLayer(6, 8, 10).forward(h, edge_index, edge_attr).shape == torch.Size(
    [4, 8]
)

##### Задание 1.2 (3 балла). E(3)-инвариантная графовая сеть

Реализуйте модель на основе реализованного вами слоя, которая принимает на вход `AtomicInterfaceGraph` и возвращает предсказанную свободную энергию связывания

Отличия от модели с практики минимальны: нужно только преобразовать расстояния с помощью модуля `RadialBasisExpansion` и передать их в каждый `InvariantLayer` вместе с очередными эмбеддингами вершин.

Модуль `RadialBasisExpansion` преобразует значения межатомных расстояний в вектор со значениями в [0, 1] с помощью набора радиальных базисных функций. Подумайте, почему такой способ обработки количественных признаков может работать лучше?

In [ ]:
class RadialBasisExpansion(nn.Module):
    offset: Tensor

    def __init__(
        self,
        start: float = 0.0,
        stop: float = 5.0,
        num_gaussians: int = 32,
    ):
        super().__init__()
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item() ** 2
        self.register_buffer("offset", offset)

    def forward(self, dist: Tensor) -> Tensor:
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))


# пример использования
dist = torch.tensor([0.1, 1.4, 2.2, 3.5, 4.4])
RadialBasisExpansion(num_gaussians=5).forward(dist).round(decimals=3)

tensor([[0.9970, 0.6550, 0.1580, 0.0140, 0.0000],
        [0.5340, 0.9930, 0.6790, 0.1710, 0.0160],
        [0.2130, 0.7490, 0.9720, 0.4640, 0.0810],
        [0.0200, 0.1980, 0.7260, 0.9800, 0.4870],
        [0.0020, 0.0420, 0.3150, 0.8740, 0.8910]])

In [ ]:
class InvariantGNN(nn.Module):
    def __init__(
        self,
        node_vocab_size: int,  # кол-во типов вершин, например атомов
        node_dim: int,  # размерность эмбеддинга вершины
        edge_dim: int,  # размерность эмбеддинга ребра
        n_layers: int,  # кол-во графовых слоёв
        dropout: float = 0.0,  # dropout rate
    ) -> None:
        super().__init__()
        ...

    def forward(self, batch: AtomicInterfaceGraph) -> Tensor:
        ...

Минимальный тест:

In [11]:
graph = Data(
    atoms=torch.randint(10, size=(4,)),
    coordinates=torch.randn(4, 3),
    edge_index=edge_index,
    is_receptor=torch.tensor([0, 0, 1, 1]),
    batch=torch.tensor(
        [0, 0, 1, 1]
    ),  # у нас 2 графа — значит, должно быть 2 числа на выходе
)
assert InvariantGNN(10, 4, 4, 2, 0.1).forward(graph).shape == torch.Size([2, 1])

#### Задание 2 (4 балла + бонусы за точность). Обучение модели

Обучите реализованную модель, выведите в конце обучения метрики на тестовой выборке (корреляции Пирсона и Спирмена).

Ваша задача: добиться корреляции Пирсона не ниже 0.4

Бонусы:
-  4 балла: за корреляцию Пирсона не ниже 0.5
-  0.5 балла за каждые следующие 0.01, т.е. за корреляцию Пирсона 0.6 вы получите 5 + 4 + 0.5 * 10 = 14 баллов


Вы можете использовать любые параметры построения графа (`graph_radius`, `n_neighbors`, `remove_intermolecular_edges`), любой размер модели и способ и настройки регуляризации, любой оптимизатор

In [12]:
train_dataset = AtomicGraphDataset(
    Path("../datasets/affinity_train.json"),
    graph_radius=6.0,
    n_neighbors=12,
    remove_intermolecular_edges=True,
)
test_dataset = AtomicGraphDataset(
    Path("../datasets/affinity_test.json"),
    graph_radius=6.0,
    n_neighbors=12,
    remove_intermolecular_edges=True,
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [13]:
torch.manual_seed(42)

model = InvariantGNN(
    node_vocab_size=len(ATOMS_INDICES) + 1,
    node_dim=32,
    edge_dim=32,
    n_layers=3,
    dropout=0.3,
)
optim = Adam(model.parameters(), lr=0.001, weight_decay=1.0, betas=(0.9, 0.999))

In [14]:
for i in range(50):
    model.train()
    for batch in train_loader:
        yhat = model.forward(batch)
        loss = F.mse_loss(yhat.flatten(), batch.affinity)
        # loss = F.huber_loss(yhat.flatten(), batch.affinity)
        loss.backward()
        optim.step()
        optim.zero_grad()

    if (i + 1) % 5 == 0:
        validate(test_loader, model)

Loss: 0.3043, Pearson R: 0.3637, Spearman R: 0.3359
Loss: 0.1240, Pearson R: 0.3859, Spearman R: 0.3583
Loss: 0.2321, Pearson R: 0.4041, Spearman R: 0.3773
Loss: 0.1667, Pearson R: 0.4272, Spearman R: 0.3851
Loss: 0.1610, Pearson R: 0.4406, Spearman R: 0.3937
Loss: 0.2147, Pearson R: 0.4588, Spearman R: 0.4030
Loss: 0.1087, Pearson R: 0.4621, Spearman R: 0.4070
Loss: 0.1406, Pearson R: 0.4710, Spearman R: 0.4091
Loss: 0.2007, Pearson R: 0.4852, Spearman R: 0.4166
Loss: 0.1427, Pearson R: 0.4911, Spearman R: 0.4285


#### Задание 3 (3 балла + бонусы за точность). В погоне за точностью

Используйте любую графовую архитектуру (кроме реализованной в задании 2 и полной копии модели с практики!), чтобы добиться корреляции Пирсона больше 0.55.

Баллы за задание:
- 3 балла — за корреляцию Пирсона выше 0.55
- +3 балла — за корреляцию Пирсона выше 0.6
- +1 балл за каждые следующие 0.01

Задание с полной свободой творчества, можно менять и архитектуру модели, и использовать любые модули из `pytorch-geometric`, и менять способ представления данных. Вот лишь некоторые идеи, которые можно тестировать:
1. **Использование аминокислотного графа**: скорее всего, если использовать только аминокислотный граф вместо полноатомного, точность существенно просядет, но не исключено, что его использование в качестве дополнительного набора признаков позволит несколько улучшить качество. Например, у вас может быть две графовых сети: олна обрабатывает полноатомный интерфейс, вторая — аминокислотный граф всей структуры, в конце вы получаете эмбеддинги этих двух графов и на их основе предсказываете изменение свободной энергии
2. **Модификация реализованной модели**: тут много вариантов, например
   - добавить линейный слой / перцептрон, который будет в каждом графовом слое преобразовывать эмбеддинг рёбер
   - изменить метод `message`, чтобы иначе формировать сообщения
   - изменить метод `update`, чтобы использовать более гибкий метод агрегации сообщений от соседей; например, реализовать механизм внимания, как в `torch_geometric.nn.conv.GATConv` 
   - добавление эмбеддингов аминокислот к эмбеддингам атомов
3. **Модификация модели с практики**: она является достаточно сильным бейзлайном, поэтому может иметь смысл поколдовать над ней: поменять гиперпараметры, функции активации, используемую функцию ошибки (например huber loss или log-cosh)